# Decision Tree from scratch

| Neighbourhood | # of rooms | Affordable (boolean) |
|----------|----------|----------|
| West   | 3 | <span style="color: green;">Yes</span> |
| West   | 5 | <span style="color: green;">Yes</span> |
| West   | 2 | <span style="color: green;">Yes</span> |
| East   | 3 | <span style="color: green;">Yes</span> |
| East   | 4 | <span style="color: green;">Yes</span> |
| East   | 6 | <span style="color: red;">No</span> |
| East   | 5 | <span style="color: red;">No</span> |
| East   | 2 | <span style="color: green;">Yes</span> |

<img src = './files/decision_tree.png'/>

What needs to be decided on?

- Split feature
- Split Point
- When to stop splitting

Steps


- Training:
    - Given the whole dataset:
        - Calculate information gain with each possible split
        - Divide set with that feature and value that gives the most information gain (IG)
        - Divide tree and do the same for all created branches...
        - ... until a stopping criteria is reached

- Testing: 
    - Given a data point:
        - Follow the tree until you reach a leaf node
        - Return the most common class label

## Theory and Terms

### Information Gain:
$$
\text{IG} = E(\text{parent}) - [\text{weighted average}] * E(\text{children})
$$
- where $E$ is called Entropy

### Entropy 

Entropy in a decision tree is a measure of the impurity or disorder within a set of data. It helps determine how a decision tree splits the data at each node to create branches that lead to the most homogeneous subsets possible.

### Key Points about Entropy:
1. **Definition**: Entropy quantifies the amount of uncertainty or impurity in a dataset. In the context of decision trees, it measures how mixed the classes are within a node.
2. **Calculation**: The entropy \( H \) of a dataset is calculated using the formula:
   $$
   E(S) = - \sum_{i=1}^{c} p_i \log_2(p_i)
   $$
   where \( S \) is the dataset, \( c \) is the number of classes, and \( p_i \) is the proportion of instances belonging to class \( i \).

3. **Purpose**: By calculating entropy, a decision tree algorithm can decide the best attribute to split the data. The goal is to choose splits that minimize entropy, leading to more pure (homogeneous) subsets¹².
4. **Information Gain**: When a dataset is split based on an attribute, the change in entropy is called information gain. The attribute with the highest information gain is chosen for the split, as it provides the most significant reduction in uncertainty.

### Stopping criteria

    - maximum depth
    - minimum number of samples
    - min impurity decrease

## Python Implementation

In [1]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        



### Dataset

In this example we use the Breast Cancer Wisconsin dataset, which is commonly used for binary classification tasks. Here are some key details:

- **Dataset Characteristics**:
  - **Classes**: 2 (malignant and benign)
  - **Samples per class**: 212 (malignant), 357 (benign)
  - **Total samples**: 569
  - **Dimensionality**: 30 features
  - **Feature types**: Real, positive values
  
- **Return Values**:
  - **data**: The data matrix (569 samples, 30 features).
  - **target**: The classification target (569 samples).
  - **feature_names**: Names of the dataset columns.
  - **target_names**: Names of the target classes.
  - **DESCR**: Full description of the dataset.
  - **filename**: Path to the location of the data file.


This dataset is great for practicing classification algorithms and understanding the basics of machine learning.


In [2]:
from sklearn import datasets
import pandas as pd

bc = datasets.load_breast_cancer()
# columns = bc.columns
X, y = bc.data, bc.target
# df = pd.DataFrame(X)

# Get the column names
column_names = bc.feature_names
label_names = bc.target_names
print(f'Features: {column_names}')
print(f'Target: {label_names}')
# Convert column names to DataFrame
df = pd.DataFrame(X, columns=column_names)
df['target'] = y
df


Features: ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
Target: ['malignant' 'benign']


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


### Train Test Split

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)


### Run the Algorithm

In [4]:

clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, predictions)
print(acc)

0.9210526315789473


Resources:

- https://www.youtube.com/watch?v=NxEHSAfFlK8